Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/production-deploy-to-aks-gpu/production-deploy-to-aks-gpu.png)

# Deploying a web service hosted on NVIDIA Triton to Azure Kubernetes Service (AKS)
This notebook shows the steps for deploying a service with [NVIDIA Triton Inferencing Server](https://developer.nvidia.com/nvidia-triton-inference-server): registering a model, creating an image, provisioning a cluster (one time action), and deploying a service to it. 
We then test and delete the service, image and model.

In [ ]:
import azureml.core
print(azureml.core.VERSION)

# Get workspace
Load existing workspace from the config file info.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Download the model

Prior to registering the model, you should have a model in one of the [supported formats](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html#framework-model-definition) by Triton. This cell will download a [pretrained ONNX densenet](https://contentmamluswest001.blob.core.windows.net/content/14b2744cf8d6418c87ffddc3f3127242/9502630827244d60a1214f250e3bbca7/08aed7327d694b8dbaee2c97b8d0fcba/densenet121-1.2.onnx) model.

** Note: ** If you have a previously-registered model, the file name needs to follow the [naming convention](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html?highlight=model%20onnx#framework-model-definition) or be specified using model configuration file below.

In [ ]:
import os
import requests
import shutil
import tarfile
import tempfile

from io import BytesIO

model_url = 'https://contentmamluswest001.blob.core.windows.net/content/14b2744cf8d6418c87ffddc3f3127242/9502630827244d60a1214f250e3bbca7/08aed7327d694b8dbaee2c97b8d0fcba/densenet121-1.2.onnx'

version = '1'
model_dir = os.path.join('models', 'triton', 'densenet_onnx', version)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

target_file = os.path.join(model_dir, 'model.onnx')

if not os.path.exists(target_file):
    response = requests.get(model_url)
    open(target_file, 'wb').write(response.content)


# Add Model Configuration file

Each Triton model needs a [Model Configuration](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_configuration.html) that provides required and optional information about the model. We've provided one in this directory already.


# Register the model
Register an existing trained model, add description and tags.

** Note: ** Under `model_path` there must be a sub-directory named `triton`, which has the structure of a Triton [Model Repository](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html#repository-layout).

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path="models", # This points to the local directory to upload.
                       model_name="densenet_onnx", # This is the name the model is registered as.
                       tags={'area': "Image classification", 'type': "classification"},
                       description="Image classification trained on Imagenet Dataset",
                       workspace=ws)

print(model.name, model.description, model.version)

# Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your GPU cluster
gpu_cluster_name = "aks-gpu-cluster"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print("Found existing gpu cluster")
except ComputeTargetException:
    print("Creating new gpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AksCompute.provisioning_configuration(
        cluster_purpose=AksCompute.ClusterPurpose.DEV_TEST,
        agent_count=1,
        vm_size="Standard_NC6",
        location="westus2")
    
    # Create the cluster with the specified name and configuration
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

    # Wait for the cluster to complete, show the output log
    gpu_cluster.wait_for_completion(show_output=True)

# Deploy the model as a web service to AKS

First create a scoring script. You can see the one we created for you in the `scripts` directory.

** Note: ** Triton server listens to a fixed local port. You may choose to use the Triton Python [client library](https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/client_library.html) to talk to it, while keeping the flexibility of pre-/post- processing.

** Note: ** If you use the Triton Python client, download the Wheels and include them in the `Environment`.

In [ ]:
triton_client_url = 'https://github.com/NVIDIA/triton-inference-server/releases/download/v2.1.0/v2.1.0_ubuntu1804.clients.tar.gz'

client_filename = 'tritonclientutils-2.1.0-py3-none-any.whl'
clientutils_filename = 'tritonhttpclient-2.1.0-py3-none-any.whl'
target_folder = 'python_client'

if not os.path.exists(target_folder):
    response = requests.get(triton_client_url)
    archive = tarfile.open(fileobj=BytesIO(response.content))
    with tempfile.TemporaryDirectory() as temp_folder:
        archive.extractall(temp_folder)
        os.mkdir(target_folder)
        shutil.copy(os.path.join(temp_folder, 'python', client_filename), target_folder)
        shutil.copy(os.path.join(temp_folder, 'python', clientutils_filename), target_folder)


Now create the deployment configuration objects and deploy the model as a webservice.

In [ ]:
# Set the web service configuration (using default here)
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment

env = Environment("deploytocloudenv")
conda_dep = CondaDependencies("env.yml")
client_whl_url = Environment.add_private_pip_wheel(workspace=ws, file_path = os.path.join(target_folder, client_filename), exist_ok=True)
clientutils_whl_url = Environment.add_private_pip_wheel(workspace=ws, file_path = os.path.join(target_folder, clientutils_filename), exist_ok=True)
conda_dep.add_pip_package(client_whl_url)
conda_dep.add_pip_package(clientutils_whl_url)
env.python.conda_dependencies = conda_dep

# TODO: replace with MCR image name
#env.docker.base_image = DEFAULT_GPU_IMAGE
env.docker.base_image = "delliwscentrec595b96.azurecr.io/azureml-tritonserver:20.07-py3"
env.docker.base_image_registry.address = "delliwscentrec595b96.azurecr.io"
env.docker.base_image_registry.username = "delliwscentrec595b96"
env.docker.base_image_registry.password = "eOlKQlx+tFa+J3Ml5a86fOHf8i=hi/ec"

# Optionally specify a worker count to leverage the capability of concurrency and server-side batching from Triton
# env.environment_variables = {"WORKER_COUNT":"128"}

inference_config = InferenceConfig(entry_script="score.py", source_directory="scripts", environment=env)
aks_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, gpu_cores=1)

# # Enable token auth and disable (key) auth on the webservice
# aks_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4, gpu_cores = 1, token_auth_enabled=True, auth_enabled=False)

In [ ]:
%%time
aks_service_name ='gpu-densenet-onnx'

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=gpu_cluster)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

# Test the web service
We test the web sevice by passing the test images content.

In [ ]:
%%time
import requests

# if (key) auth is enabled, fetch keys and include in the request
key1, key2 = aks_service.get_keys()

headers = {'Content-Type':'application/octet-stream', 'Authorization': 'Bearer ' + key1}

# # if token auth is enabled, fetch token and include in the request
# access_token, fetch_after = aks_service.get_token()
# headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

test_sample = open('car.jpg', 'rb').read()
resp = requests.post(aks_service.scoring_uri, test_sample, headers=headers)
print(resp.text)

# Clean up
Delete the service, image, model and compute target

In [ ]:
%%time
aks_service.delete()
model.delete()
gpu_cluster.delete()


In [ ]:
aks_service.wait_for_deployment(True)